# COVID19 and Pneumonia detection from chest xray: Deep learning using fastai

This notebook combines two different datasets [1,2] on chest xray and build a model to distinguish between Normal, Pneumonia and COVID19 chest xrays. The work here is very similar to the nice work of Adrian Rosebrock [3] with the following diffrences:
1. Including Pneumonia images to predict between 3 classes: Normal, Pneumonia, COVID
2. Using all available dataset: [3] keeps the numver of samples equal by undersampling. But this notebook includes all available data which leads to a highly imblanced dataset at the moment. However, when there will be more data available in [2], this would be less of an issue.
3. Using fastai and Pytorch instead of Keras and TensorFlow

[1] http://dx.doi.org/10.17632/rscbjbr9sj.2#file-41d542e7-7f91-47f6-9ff2-dd8e5a5a7861

[2] https://github.com/ieee8023/covid-chestxray-dataset/tree/master/images

[3] https://www.pyimagesearch.com/2020/03/16/detecting-covid-19-in-x-ray-images-with-keras-tensorflow-and-deep-learning/


Disclaimer: The available COVID dataset (at the moment) is not consistent and has been pulled from different studies. Thus, be careful with interpretation of the results.

Acknowledge: https://course.fast.ai/

## Import libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import os
import shutil
from fastai.vision import *
from fastai.metrics import error_rate

## Combine dataset from two different repositories

In [ ]:
# create dataset folder with following subfolders:
#.     NORMAL/
#.     PNEUMONIA/
#.     COVID

dataset_path = './dataset'
!mkdir dataset
!mkdir dataset/NORMAL
!mkdir dataset/PNEUMONIA
!mkdir dataset/COVID

In [ ]:
# copy NORMAL and PNEUMONIA xrays from chest-xray-pneumonia repository
!cp ../input/chest-xray-pneumonia/chest_xray/train/NORMAL/* dataset/NORMAL/
!cp ../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/* dataset/PNEUMONIA/

In [ ]:
# copy COVID xrays from covid-chest-xray repository

covid_dataset_path = '../input/covid-chest-xray'
csvPath = os.path.sep.join([covid_dataset_path, "metadata.csv"])
df = pd.read_csv(csvPath)

for (i, row) in df.iterrows():
    if row["finding"] != "COVID-19" or row["view"] != "PA":
        continue

    imagePath = os.path.sep.join([covid_dataset_path, "images", row["filename"]])

    if not os.path.exists(imagePath):
        continue

    filename = row["filename"].split(os.path.sep)[-1]
    outputPath = os.path.sep.join(["dataset/COVID", filename])
    shutil.copy2(imagePath, outputPath)

## Load data in ImageDataBunch format of fastai

In [ ]:
path = Path("./dataset")
data = ImageDataBunch.from_folder(path, ds_tfms=get_transforms(), valid_pct=0.2, size=256).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=10, figsize=(10,10))

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=[accuracy, error_rate, Recall(average='macro'), Precision(average='macro')])

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot(suggestion=True)
min_grad_lr = learn.recorder.min_grad_lr
learn.fit_one_cycle(5, min_grad_lr)

In [ ]:
learn.save('after_unfreezing')

## Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(6,6), dpi=60)
interp.plot_top_losses(16, figsize=(15,11))